In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
p = "../DigitDataset/"
dir = os.listdir(p)
df = pd.DataFrame(columns =['path','label'])

path_to_data = []
class_name = []
for el in dir:
    if os.path.isdir(p+el):
        DIR = os.listdir(p+el)
        for label in DIR:
            path_to_data.append(p+el+"/"+label)
            class_name.append(el)
            
s = set(class_name)
df = pd.DataFrame(data={'path':path_to_data, 'label':class_name})

sh = df.shape[0]
array_a = np.random.permutation(range(sh))
id_train = round(0.8*sh)
id_test = round(0.1*sh)
id_val = round(0.1*sh)

train_idx = array_a[0:id_train]
df_train = df.iloc[train_idx,:]
val_idx = array_a[id_train+1:id_train+id_val]
df_val = df.iloc[val_idx,:]
test_idx = array_a[id_train+id_val+1:-1]
df_test = df.iloc[test_idx,:]

from torch.utils.data import Dataset
s = list(s)

class imgDataset(Dataset):
    def __init__(self,df,s=s, transform=None):
        self.df = df
        self.class_names = s
        if transform is None:
            self.should_transform = False
        else:
            self.transform = transform
            self.should_transform = True
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        label = self.df.iloc[idx, 1]
        # print(self.class_names)
       # lab_class = self.df.iloc[idx, 1]
        label = self.class_names.index(label)
       # label = label[0]
        image = plt.imread(self.df.iloc[idx,0])
       # image = np.array(image)
        image = np.swapaxes(image, -1, 0)
       # image = torch.tensor(image, dtype=torch.float32)
       # print(image)
        if self.should_transform:
            image = self.transform(image)          
        else:
            image = image
            
        return {"image": image, "label": label} 
  

from torchvision import transforms 

    
data_transform_list = [transforms.ToPILImage(),
                       transforms.ToTensor()]
data_transf_val = [transforms.ToPILImage(),
                       transforms.ToTensor()]

transform = transforms.Compose(data_transform_list)

transform_val = transforms.Compose(data_transf_val)
dataset = imgDataset(df_train,transform=transform)

dataset_val = imgDataset(df_val,transform=transform_val)


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
import torch.nn as nn
import torch.nn.functional as F
torch.backends.cudnn.benchmarks = True #cudnn
#torch.backends.cudnn.allow_tf32 #cudnn
#==========================
# добавить слой нормировки по батчу
# добавить еще один слой свертки и сравнить результаты с исходной реализацией

class ff_net(nn.Module):
    def __init__(self):
        super(ff_net, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output 
    
model = ff_net()
 
criterion = nn.CrossEntropyLoss()


In [ ]:
from torch.utils.data import DataLoader
data_load = DataLoader(dataset,batch_size=32, shuffle=True, pin_memory=True) 


In [ ]:
import torch.optim as optim
from torch.autograd import Variable

NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(NUM_EPOCHS):
    test_error_count = 0.0
    #for images, labels in iter(data_load):
    for i, batch in enumerate(data_load):
        # images, labels = next(iter(dataloader))
        images = batch['image']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        test_error_count += float(torch.sum((labels == outputs.argmax(1))))
        
    # =============================================================================
    #     добавить код тестирования на валидации в процессе обучения
    #     
    #         
    #         
    #         
    #         
    # =============================================================================

    test_accuracy = 1.0 - float(test_error_count) / float(len(dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy  